In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

In [3]:
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(200, height), (1100, height), (550,250)]]) 
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [4]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:   
            cv2.line(line_image, (x1, y1), (x2, y2), (0,0,255), 10)
    return line_image

In [5]:
def make_coordinates(image, line_parameters):
    try:
        slope, intercept = line_parameters
    except TypeError:
        slope, intercept = 0.0001,0
    #slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*3/5)
    x1 = int((y1 -  intercept)/slope)
    x2 = int((y2 -  intercept)/slope)
    return np.array([x1, y1, x2, y2])

In [6]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2), (y1,y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope <0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
#     if left_fit:
#         left_fit_average = np.average(left_fit, axis=0)
#         #print(left_fit_average, 'left')
#         left_line = make_coordinates(image, left_fit_average)
#     if right_fit:
#         right_fit_average = np.average(right_fit, axis=0)
#         #print(right_fit_average, 'right')
#         right_line = make_coordinates(image, right_fit_average)
    left_fit_avg = np.average(left_fit, axis = 0)
    right_fit_avg = np.average(right_fit, axis = 0)
    left_line = make_coordinates(image, left_fit_avg)
    right_line = make_coordinates(image, right_fit_avg)
    return np.array([left_line, right_line])

In [7]:
image = cv2.imread('test_image.jpeg')
lane_image = np.copy(image)
canny_image = canny(lane_image)
cropped_image = region_of_interest(canny_image)
lines= cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100,
                       np.array([]), minLineLength = 40, maxLineGap = 5)
averages_lines= average_slope_intercept(lane_image, lines)
line_image = display_lines(lane_image, averages_lines)
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)
cv2.imshow('lane', combo_image)
cv2.imwrite('out_image.jpeg', combo_image)
cv2.waitKey(0)

113

In [8]:
cap = cv2.VideoCapture('test_vid.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('out_vid.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 40, (frame_width,frame_height))

while(cap.isOpened()):
    ret , frame = cap.read()
    if ret == True:
        canny_image = canny(frame)
        cropped_image = region_of_interest(canny_image)
        lines= cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100,
                               np.array([]), minLineLength = 40, maxLineGap = 5)
        averages_lines= average_slope_intercept(frame, lines)
        line_image = display_lines(frame, averages_lines)
        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        cv2.imshow('lane', combo_image)
        
        out.write(combo_image)


        if cv2.waitKey(2) & 0xFF == ord('q'):
            break
    else:
        break
            
cap.release()
out.release()
cv2.destroyAllWindows()

/Users/kazmi/miniforge3/envs/tfexp/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/kazmi/miniforge3/envs/tfexp/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
#plt.imshow(canny_image)